<a href="https://colab.research.google.com/github/sr4rish/FakeNewsClassfication-LSTM/blob/master/FakeNewsClassifierLatest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import tensorflow as tf
import re
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer

In [41]:
df=pd.read_csv('/content/drive/My Drive/FakeNewsClassifier/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [42]:
df.shape

(20800, 5)

In [43]:
# checking for missing values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [44]:
df_train = df[['title','label']]
df_train.head()

,title,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [45]:
# checking for missing values
df_train.isnull().sum()

title    558
label      0
dtype: int64

In [46]:
###Drop Nan Values
df_train=df_train.dropna()

In [47]:
length = []
[length.append(len(str(text))) for text in df_train['title']]
df_train['length'] = length
df_train.head()

,title,label,length
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,81
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,55
2,Why the Truth Might Get You Fired,1,33
3,15 Civilians Killed In Single US Airstrike Hav...,1,63
4,Iranian woman jailed for fictional unpublished...,1,93


In [48]:
df_train.title[4]

'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'

In [49]:
#tracking outliers and max feature
min(df_train['length']), max(df_train['length']), round(sum(df_train['length'])/len(df_train['length']))

(3, 456, 74)

In [50]:
len(df_train[df_train['length'] < 200])

20208

In [51]:
df_train['title'][df_train['length'] < 10]

86          Poison
1977     Tweetwave
2561          Wow!
2881     6.1 Italy
5697      Trumped!
7423       俄罗斯总统观注
10596    Rick Rule
11624    What If….
12617     Untitled
15728          Rum
16049    What Now?
17081         EPIC
17607    Candidate
17927    Wikileaks
19428     Ooh Fuck
19741    RIP, Vine
20523        White
Name: title, dtype: object

In [52]:
df_train['title'][df_train['length'] > 200]

317      Life: If You Love Enamel Pins, You’ll Love Thi...
569      Comment on Architecture is More Than a Clever ...
1173     Roger Stone update on Stop the Steal exit poll...
2095     6-minute analysis: Will FBI agents break ‘Empe...
3314     24-minute game-changing video: Electronic ‘vot...
3397     Our Landfill Economy: Want to make a quick pro...
3419     Comment on These Lifestyle Choices Lower Your ...
3563     Comment on 10 Presidents & Politicians Who Tol...
4166     Comment on BBC targets children to promote the...
5217     Comment on Russian Government Calls For Intern...
5267     12 and 63-minute videos: Donald Trump connects...
5773     Dr. Jim Willie: ‘The Western central bank fran...
6395     Comment on ‘Voting Is Rigged & Used To Pacify ...
8717     Secret Emails From Hillary Clinton Reveal: The...
8998     1-hour video: Bush-Clinton and .01% ‘Deep Stat...
9858     Comment on Guess what happened in Chiraq this ...
10448    Comment on More Than 60 School Children Witnes.

In [53]:
# dropping the outliers
df_train = df_train.drop(df_train['title'][df_train['length'] < 10].index, axis = 0)
df_train = df_train.drop(df_train['title'][df_train['length'] > 200].index, axis = 0)
df_train.head()

,title,label,length
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,81
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,55
2,Why the Truth Might Get You Fired,1,33
3,15 Civilians Killed In Single US Airstrike Hav...,1,63
4,Iranian woman jailed for fictional unpublished...,1,93


In [54]:
len(df_train['length'])

20191

In [55]:
min(df_train['length']), max(df_train['length'])

(10, 199)

In [56]:
max_features = 5000
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = df_train['title'])
X = tokenizer.texts_to_sequences(texts = df_train['title'])

In [57]:
# now applying padding to make them even shaped.
sent_length = 30
X = pad_sequences(sequences = X, maxlen = sent_length, padding = 'pre')

In [58]:
## Get the Dependent features
y=df_train['label'].values
print(X.shape)
print(y.shape)

(20191, 30)
(20191,)


In [59]:
X[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        887,   16,   18,  151,  191,   13, 1224,   12], dtype=int32)

In [60]:
## Creating model
embedding_vector_features=64
model=Sequential()
model.add(Embedding(max_features,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.3))
#model.add(Dense(120,activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 64)            320000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 386,101
Trainable params: 386,101
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
# splitting the data training data for training and validation.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [62]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=124)

Epoch 1/10
131/131 [==============================] - 9s 68ms/step - loss: 0.3452 - accuracy: 0.8486 - val_loss: 0.1515 - val_accuracy: 0.9413
Epoch 2/10
131/131 [==============================] - 9s 66ms/step - loss: 0.1175 - accuracy: 0.9563 - val_loss: 0.1448 - val_accuracy: 0.9440
Epoch 3/10
131/131 [==============================] - 9s 66ms/step - loss: 0.0798 - accuracy: 0.9724 - val_loss: 0.1438 - val_accuracy: 0.9421
Epoch 4/10
131/131 [==============================] - 9s 66ms/step - loss: 0.0575 - accuracy: 0.9817 - val_loss: 0.1622 - val_accuracy: 0.9361
Epoch 5/10
131/131 [==============================] - 9s 66ms/step - loss: 0.0379 - accuracy: 0.9875 - val_loss: 0.1841 - val_accuracy: 0.9393
Epoch 6/10
131/131 [==============================] - 9s 66ms/step - loss: 0.0240 - accuracy: 0.9931 - val_loss: 0.2574 - val_accuracy: 0.9411
Epoch 7/10
131/131 [==============================] - 9s 66ms/step - loss: 0.0150 - accuracy: 0.9959 - val_loss: 0.3136 - val_accuracy: 0.9379

In [63]:
from sklearn.metrics import confusion_matrix

In [66]:
y_pred=model.predict_classes(X_test)

In [67]:
confusion_matrix(y_test,y_pred)

array([[1963,  175],
       [  73, 1828]])

In [68]:
model.save("/content/drive/My Drive/FakeNewsClassifier/FakeNewsModel(New).h5")